In [77]:
import os
import csv
import shutil
import matplotlib.pyplot
import pickle
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras import optimizers
from keras.applications import VGG16
from PIL import Image



In [80]:


#create strings for directory access
baseDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset'

finalTrainDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\FINALTRAIN'

testDirectory = r'C:\Users\Administrator\Desktop\Personal-Projects-Blood-Type-Prediction\Dataset\TEST'

subdirectoriesList = [ 'EOSINOPHIL', 'LYMPHOCYTE', 'MONOCYTE', 'NEUTROPHIL' ]



In [31]:


#make small train directories because the learning time is too long

# os.chdir( baseDirectory )
# os.mkdir( 'SMALLTRAIN' )

# os.chdir( smallTrainDirectory )
# os.mkdir( 'EOSINOPHIL' )
# os.mkdir( 'LYMPHOCYTE' )
# os.mkdir( 'MONOCYTE' )
# os.mkdir( 'NEUTROPHIL' )


# os.chdir( trainDirectory )
# for subdirectory in subdirectoriesList:
#     os.chdir( trainDirectory + '\\' + subdirectory )
#     fileNames = os.listdir()
    
#     for fileNameIndex in range( 600 ):
#         fileName = fileNames[ fileNameIndex ]
#         shutil.copy( fileName, smallTrainDirectory + '\\' + subdirectory )
        
    
    

#make small test directory because the learning time is too long
# os.chdir( baseDirectory )
# os.mkdir( 'SMALLTEST' )

# os.chdir( smallTestDirectory )
# os.mkdir( 'EOSINOPHIL' )
# os.mkdir( 'LYMPHOCYTE' )
# os.mkdir( 'MONOCYTE' )
# os.mkdir( 'NEUTROPHIL' )


# os.chdir( testDirectory )
# for subdirectory in subdirectoriesList:
#     os.chdir( testDirectory + '\\' + subdirectory )
#     fileNames = os.listdir()
    
#     for fileNameIndex in range( 250 ):
#         fileName = fileNames[ fileNameIndex ]
#         shutil.copy( fileName, smallTestDirectory + '\\' + subdirectory )



In [83]:

#create generators for images
finalTrainGenerator = ImageDataGenerator(rescale = (1/255))

testGenerator = ImageDataGenerator(rescale = (1/255))



finalTrainDirectoryIterator = finalTrainGenerator.flow_from_directory(directory = finalTrainDirectory,
                                                                      target_size = (128, 128),
                                                                      class_mode = 'categorical',
                                                                      batch_size = 128)


testDirectoryIterator = testGenerator.flow_from_directory(directory = testDirectory,
                                                          target_size = (128, 128),
                                                          class_mode = 'categorical',
                                                          batch_size = 128)




Found 9957 images belonging to 4 classes.
Found 2487 images belonging to 4 classes.


In [63]:

#create the convolutional base 
convolutionalBase = VGG16( weights = 'imagenet',
                           include_top = False,
                           input_shape = ( 128, 128, 3 ) )


#freeze the convolutional base
convolutionalBase.trainable = False 


#create the sequential model with the VGG16 convolutional base
VGG16SequentialModel = Sequential()

VGG16SequentialModel.add( convolutionalBase )

VGG16SequentialModel.add( Flatten() )

VGG16SequentialModel.add( Dropout( 0.3 ) )

VGG16SequentialModel.add( Dense( 256, activation = 'relu' ) )

VGG16SequentialModel.add( Dense( 4, activation = 'softmax' ) )


#compile the model
VGG16SequentialModel.compile( optimizer = optimizers.RMSprop( lr = 1e-4 ),
                           loss = 'categorical_crossentropy',
                           metrics = [ 'acc' ] )


#train the model
VGG16SequentialModelHistory = VGG16SequentialModel.fit_generator( generator = finalTrainDirectoryIterator,
                                                                 steps_per_epoch = 78,
                                                                 epochs = 80,
                                                                 verbose = True,
                                                                 validation_data = testDirectoryIterator,
                                                                 validation_steps = 20 )

#save the model
sequentialModel.save( 'BloodTypeClassifierUsingVGG16Base.h5' )



Epoch 1/100
78/78 [==============================] - 36s 462ms/step - loss: 1.3046 - acc: 0.3904 - val_loss: 1.2310 - val_acc: 0.4202
Epoch 2/100
78/78 [==============================] - 35s 445ms/step - loss: 1.0904 - acc: 0.5296 - val_loss: 1.2169 - val_acc: 0.4660
Epoch 3/100
 5/78 [>.............................] - ETA: 26s - loss: 1.0398 - acc: 0.5453

KeyboardInterrupt: 

In [ ]:
#retrieve the training history
VGG16SequentialModelHistory = VGG16SequentialModelHistory.history

#save the training history to file
os.chdir( baseDirectory )

with open('VGG16TrainHistoryDictionary.obj', 'wb') as fileToPickle:
        pickle.dump(VGG16SequentialModelHistory, fileToPickle)


In [84]:
#Create a sequential model from conv2D layers
fromScratchSequentialModel = Sequential()

fromScratchSequentialModel.add( Conv2D( 32, (3,3), activation = 'relu', input_shape = (128, 128, 3) ) )

fromScratchSequentialModel.add( MaxPooling2D( (2,2) ) )
                               
fromScratchSequentialModel.add( Conv2D( 64, (3,3), activation = 'relu' ) )

fromScratchSequentialModel.add( MaxPooling2D( (2,2) ) )
                               
fromScratchSequentialModel.add( Conv2D( 128, (3,3), activation = 'relu' ) )

fromScratchSequentialModel.add( MaxPooling2D( (2,2) ) )
                               
fromScratchSequentialModel.add( Conv2D( 128, (3,3), activation = 'relu' ) )

fromScratchSequentialModel.add( MaxPooling2D( (2,2) ) )
                               
fromScratchSequentialModel.add( Flatten() )

fromScratchSequentialModel.add( Dropout( 0.3 ) )

fromScratchSequentialModel.add( Dense( 512, activation = 'relu' ) )

fromScratchSequentialModel.add( Dense( 4, activation = 'softmax' ) )
                               

#compile the model
fromScratchSequentialModel.compile( optimizer = optimizers.RMSprop( lr = 1e-4 ),
                                   loss = 'categorical_crossentropy',
                                   metrics = [ 'acc' ] )


#train the model
fromScratchSequentialModelHistory = fromScratchSequentialModel.fit_generator( generator = finalTrainDirectoryIterator,
                                                                 steps_per_epoch = 78,
                                                                 epochs = 100,
                                                                 verbose = True,
                                                                 validation_data = testDirectoryIterator,
                                                                 validation_steps = 20 )


#save the model
fromScratchSequentialModel.save( 'BloodTypeClassifierFromScratch.h5' )


Epoch 1/80
30/78 [==========>...................] - ETA: 13s - loss: 1.3887 - acc: 0.2568

KeyboardInterrupt: 

In [ ]:

#retrieve the training history
fromScratchSequentialModelHistory = fromScratchSequentialModelHistory.history

#save the training history to file
os.chdir( baseDirectory )

with open('fromScratchTrainHistoryDictionary.obj', 'wb') as fileToPickle:
        pickle.dump(sequentialModelHistory, fileToPickle)
                               